# RAG with watsonx.ai & Watson Discovery using Python SDK

### Install Discovery SDK Import the Dependencies

In [7]:
#!pip ibm-watson-machine-learning
!pip install ibm-watson

In [33]:
import json
import requests
import ast, os
import pandas as pd
import time

from ibm_watson import DiscoveryV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

In [34]:
WX_API_KEY = "REPLACE WITH YOUR IAM APIKEY"
WX_PROJECT_ID = "REPLACE WITH YOUR WXAI PROJECT ID"
WX_URL=  "https://us-south.ml.cloud.ibm.com"

WD_API_KEY = "REPLACE WITH YOUR WATSON DISCOVERY APIKEY"
WD_PROJECT_ID = "REPLACE WITH YOUR WATSON DISCOVERY PROJECT ID"
WD_URL = "https://api.us-south.discovery.watson.cloud.ibm.com/instances/REPLACE WITH YOUR INSTANCE ID"

creds = {
    "url": WX_URL,
    "apikey": WX_API_KEY 
}

In [35]:
def wd_auth(WD_PROJECT_ID=WD_PROJECT_ID, WD_API_KEY=WD_API_KEY, WD_URL=WD_URL):
    authenticator = IAMAuthenticator(WD_API_KEY) 
    discovery = DiscoveryV2(
            version='2019-04-30',
            authenticator=authenticator
        )
    discovery.set_service_url(WD_URL)

    collections = discovery.list_collections(project_id=WD_PROJECT_ID).get_result()
    collection_list = list(pd.DataFrame(collections['collections'])['collection_id'])
    return discovery, collection_list

def query_docs(keyword, WD_PROJECT_ID=WD_PROJECT_ID):
    discovery, collection_list = wd_auth()
    query_result = discovery.query(
                project_id=WD_PROJECT_ID,
                collection_ids=collection_list,
                passages={'characters':2000},
                natural_language_query=keyword).get_result(),
    #print(query_result)
    try:
        #passage = query_result[0]['results'][0]['text'][0] #get whole text
        passage = query_result[0]['results'][0]['document_passages'][0]['passage_text'] #get n characters of passage
        passage = "  ".join([line.strip() for line in passage.split("\n") if line.strip() != ""])
    except:
        passage = "no information found"
    return passage

In [36]:
def send_to_watsonxai(prompt, creds=creds, project_id=WX_PROJECT_ID,
                    model_name='meta-llama/llama-3-70b-instruct', #'mistralai/mixtral-8x7b-instruct-v01',', #'meta-llama/llama-2-13b-chat', #
                    decoding_method="greedy",
                    max_new_tokens=300,
                    min_new_tokens=1,
                    temperature=0,
                    repetition_penalty=1.0,
                    stop_sequences=[],
                    ):
    '''
   helper function for sending prompts and params to Watsonx.ai
    
    Args:  
        prompts:list list of text prompts
        decoding:str Watsonx.ai parameter "sample" or "greedy"
        max_new_tok:int Watsonx.ai parameter for max new tokens/response returned
        temperature:float Watsonx.ai parameter for temperature (range 0>2)
        repetition_penalty:float Watsonx.ai parameter for repetition penalty (range 1.0 to 2.0)

    Returns: None
        prints response
    '''

    assert not any(map(lambda prompt: len(prompt) < 1, prompt)), "make sure none of the prompts in the inputs prompts are empty"

    # Instantiate parameters for text generation
    model_params = {
        GenParams.DECODING_METHOD: decoding_method,
        GenParams.MIN_NEW_TOKENS: min_new_tokens,
        GenParams.MAX_NEW_TOKENS: max_new_tokens,
        GenParams.RANDOM_SEED: 42,
        GenParams.TEMPERATURE: temperature,
        GenParams.REPETITION_PENALTY: repetition_penalty,
        GenParams.STOP_SEQUENCES: stop_sequences
    }

    # Instantiate a model proxy object to send your requests
    model = Model(
        model_id=model_name,
        params=model_params,
        credentials=creds,
        project_id=project_id)
    
    
    output = model.generate_text(prompt)
    return output


def send_to_watsonxai_stream(prompt, creds=creds, project_id=WX_PROJECT_ID,
                    model_name='meta-llama/llama-3-70b-instruct', #'mistralai/mixtral-8x7b-instruct-v01',', #'meta-llama/llama-2-13b-chat', #
                    decoding_method="greedy",
                    max_new_tokens=300,
                    min_new_tokens=1,
                    temperature=0,
                    repetition_penalty=1.0,
                    stop_sequences=[],
                    ):
    '''
   helper function for sending prompts and params to Watsonx.ai
    
    Args:  
        prompts:list list of text prompts
        decoding:str Watsonx.ai parameter "sample" or "greedy"
        max_new_tok:int Watsonx.ai parameter for max new tokens/response returned
        temperature:float Watsonx.ai parameter for temperature (range 0>2)
        repetition_penalty:float Watsonx.ai parameter for repetition penalty (range 1.0 to 2.0)

    Returns: None
        prints response
    '''

    assert not any(map(lambda prompt: len(prompt) < 1, prompt)), "make sure none of the prompts in the inputs prompts are empty"

    # Instantiate parameters for text generation
    model_params = {
        GenParams.DECODING_METHOD: decoding_method,
        GenParams.MIN_NEW_TOKENS: min_new_tokens,
        GenParams.MAX_NEW_TOKENS: max_new_tokens,
        GenParams.RANDOM_SEED: 42,
        GenParams.TEMPERATURE: temperature,
        GenParams.REPETITION_PENALTY: repetition_penalty,
        GenParams.STOP_SEQUENCES: stop_sequences
    }

    # Instantiate a model proxy object to send your requests
    model = Model(
        model_id=model_name,
        params=model_params,
        credentials=creds,
        project_id=project_id)
    
    output = model.generate_text_stream(prompt)
    # output = model.generate_text(prompt) # This is for not streaming
    for chunk in output:
        yield chunk

In [37]:
def query_wxai(user_question, db_selection, llm_model, streaming=False):
    print(user_question)
    start_time = time.time()

    if  db_selection == 'wd':
        passage = query_docs(user_question)
        print(passage)
    else:
        collection_name = "collection"
        passage = similarity_search(user_question, milvus_connection_alias="default", collection_name=collection_name, limit=3)
        print(passage)
    
    eta_retrieve = time.time() - start_time
    print(db_selection, " eta_retrieve: ", eta_retrieve)

    prompt = f"""Anda adalah asisten yang membantu, sopan, dan jujur. Selalu jawab sebisa mungkin dengan cara yang membantu, sambil tetap aman. Jawaban Anda tidak boleh mengandung konten yang berbahaya, tidak etis, rasialis, seksis, beracun, berbahaya, atau ilegal. Pastikan bahwa respons Anda bersifat sosial tidak memihak dan positif.
    Konteks:{passage}
    Pertanyaan:{user_question}
    Harap pahami konteksnya dan jawablah pertanyaan hanya berdasarkan informasi yang diberikan. Jawab pertanyaan dengan jelas, lengkap, informatif.
    Identifikasi dan ekstrak URL yang disebutkan dalam konteks jika berkaitan dengan pertanyaan. Jangan sertakan URL yang tidak berhubungan dalam jawaban Anda. 
    Berikan jawaban secara berurutan jika diperlukan atau berikan daftar yang jelas dan ringkas. Jika suatu pertanyaan tidak masuk akal atau tidak koheren secara faktual, jelaskan mengapa dari pada menjawab sesuatu yang tidak benar. 
    Jika Anda tidak tahu jawaban atas suatu pertanyaan, tolong jangan memberikan informasi palsu. Jika konteks tidak ada hubungan dengan pertanyaan, jawab saja tidak tahu. Jawablah hanya berdasarkan konteks.\nBerikan Jawaban dalam bahasa yang sama dengan Pertanyaan.
    Sebagai contoh, jika Pertanyaannya dalam bahasa Inggris, maka jawablah dalam bahasa Inggris; Jika Pertanyaan menggunakan Mandarin, maka tuliskan Jawaban dalam bahasa Mandarin.
    Jawaban:"""

    print("streaming =", streaming)

    if streaming:
        return send_to_watsonxai_stream(prompt, model_name=llm_model, creds=creds, project_id=WX_PROJECT_ID)

    else:
        result = send_to_watsonxai(prompt, model_name=llm_model, creds=creds, project_id=WX_PROJECT_ID)
        eta_wx = time.time() - start_time
        print(llm_model, " eta_wx: ", eta_wx-eta_retrieve)
        return result, eta_retrieve, eta_wx-eta_retrieve

In [38]:
user_question = "aturan pakaian karyawan gimana?"

query_wxai(user_question, "wd", 'meta-llama/llama-3-70b-instruct' )

aturan pakaian karyawan gimana?
Gaji setiap <em>karyawan</em> akan ditinjau oleh perusahaan secara berkala dengan  memperhatikan prestasi kerja <em>karyawan</em>. Penetapan besaran kenaikan gaji akan dilakukan  dengan mempertimbangkan keadaan, kemampuan, dan kondisi perusahaan.  d. Penentuan peraturan perpajakan terhadap pajak penghasilan menjadi tanggung jawab  setiap <em>karyawan</em> sedangkan perusahaan hanya membantu untuk mengumpulkannya untuk  diserahkan kepada pihak Pemerintah atau Kantor Pelayanan Pajak.  3. Pensiun <em>Karyawan</em>:  a. Usia pensiun <em>karyawan</em> adalah 60 tahun.  b. Dana pensiun <em>karyawan</em> yang sudah mencapai usia pensiun akan diberikan satu bulan  setelah berakhirnya hubungan kerja.  4. <em>Pakaian</em> Kerja <em>Karyawan</em>:  a. <em>Karyawan</em> diwajibkan menggunakan <em>pakaian</em> seragam setiap hari Senin dan Selasa. yang  akan diberikan oleh perusahaan sesuai dengan setiap departemen perusahaan.  b. Pada hari Rabu sampai Jumat diwajibk

(' \n    Aturan pakaian karyawan adalah sebagai berikut:\n    a. Karyawan diwajibkan menggunakan pakaian seragam setiap hari Senin dan Selasa yang akan diberikan oleh perusahaan sesuai dengan setiap departemen perusahaan.\n    b. Pada hari Rabu sampai Jumat diwajibkan menggunakan pakaian kerja berupa kemeja kasual dan celana bahan. Karyawan tidak boleh menggunakan baju kaos dan celana jeans.',
 1.34505295753479,
 6.934185266494751)

In [39]:
user_question = "aturan pakaian karyawan gimana?"

for chunk in query_wxai(user_question, "wd", 'meta-llama/llama-3-70b-instruct', streaming="True"):
    print(chunk, end='')

aturan pakaian karyawan gimana?
Gaji setiap <em>karyawan</em> akan ditinjau oleh perusahaan secara berkala dengan  memperhatikan prestasi kerja <em>karyawan</em>. Penetapan besaran kenaikan gaji akan dilakukan  dengan mempertimbangkan keadaan, kemampuan, dan kondisi perusahaan.  d. Penentuan peraturan perpajakan terhadap pajak penghasilan menjadi tanggung jawab  setiap <em>karyawan</em> sedangkan perusahaan hanya membantu untuk mengumpulkannya untuk  diserahkan kepada pihak Pemerintah atau Kantor Pelayanan Pajak.  3. Pensiun <em>Karyawan</em>:  a. Usia pensiun <em>karyawan</em> adalah 60 tahun.  b. Dana pensiun <em>karyawan</em> yang sudah mencapai usia pensiun akan diberikan satu bulan  setelah berakhirnya hubungan kerja.  4. <em>Pakaian</em> Kerja <em>Karyawan</em>:  a. <em>Karyawan</em> diwajibkan menggunakan <em>pakaian</em> seragam setiap hari Senin dan Selasa. yang  akan diberikan oleh perusahaan sesuai dengan setiap departemen perusahaan.  b. Pada hari Rabu sampai Jumat diwajibk